In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import pprint
%matplotlib inline

import os
print(os.listdir("../input"))

Using TensorFlow backend.


['2009Mahi.csv', '2010Anur.csv', '2010Mahi.csv', '2011Anur.csv', '2011Mahi.csv', '2012Mahi.csv', '2019Anur.csv', '2019Colo - Copy - Copy.csv', '2019Colo.csv', 'Anur(test).csv', 'Batt.csv', 'Colo - Cloud.csv', 'Colo - Copy (2).csv', 'Colo - Copy.csv', 'Colo - Temp.csv', 'Colo(test) - Copy.csv', 'Colo(test).csv', 'Colo.csv', 'humidity.csv', 'PrecipMM.csv', 'PrecipMM2.csv', 'PrecipMM3.csv', 'PredictedTemp.csv', 'Temp(4).csv', 'TempPythonist(1).csv', 'TempPythonist(2).csv', 'TempPythonist(3).csv', 'TempV1(2).csv', 'TempV1.csv', 'TempV2.csv', 'Temp_(2020)', 'Temp_(2020).h5']


In [2]:
df = pd.read_csv('../input/Colo - Cloud.csv')

In [4]:
df

,Date,Cloudcover
0,1/1/2009,5
1,2/1/2009,6
2,3/1/2009,4
3,4/1/2009,2
4,5/1/2009,11
...,...,...
4012,27/12/2019,29
4013,28/12/2019,40
4014,29/12/2019,8
4015,30/12/2019,23


In [5]:
df.Date=pd.to_datetime(df.Date)
df=df.set_index('Date')

In [6]:
df

,Cloudcover
Date,
2009-01-01,5
2009-02-01,6
2009-03-01,4
2009-04-01,2
2009-05-01,11
...,...
2019-12-27,29
2019-12-28,40
2019-12-29,8


In [7]:
TestData = df.tail(365)

In [8]:
Df_Total = pd.concat((df[["Cloudcover"]], TestData[["Cloudcover"]]), axis=0)

In [9]:
Df_Total.shape

(4382, 1)

In [10]:
inputs = Df_Total[len(Df_Total) - len(TestData) - 60:].values
inputs.shape

(425, 1)

In [11]:
from pandas.tseries.offsets import DateOffset
add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,366)]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [12]:
future_dates.tail(365)

,Cloudcover
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
...,...
2020-12-26,NaN
2020-12-27,NaN
2020-12-28,NaN
2020-12-29,NaN


In [13]:
df.shape

(4017, 1)

In [14]:
Training_Set = df

In [15]:
Training_Set = Training_Set.values
sc = MinMaxScaler(feature_range=(0, 1))
Train = sc.fit_transform(Training_Set)

In [16]:
Train.shape

(4017, 1)

In [17]:
X_Train = []
Y_Train = []

# Range should be fromm 60 Values to END
for i in range(60, Train.shape[0]):
   
    # X_Train 0-59
    X_Train.append(Train[i-60:i])
   
    # Y Would be 60 th Value based on past 60 Values
    Y_Train.append(Train[i])

# Convert into Numpy Array
X_Train = np.array(X_Train)
Y_Train = np.array(Y_Train)

print(X_Train.shape)
print(Y_Train.shape)

(3957, 60, 1)
(3957, 1)


In [18]:
X_Train = np.reshape(X_Train, newshape=(X_Train.shape[0], X_Train.shape[1], 1))
X_Train.shape

(3957, 60, 1)

# Model

In [19]:
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 150, return_sequences = True,input_shape = (X_Train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 150, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 150, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 150))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

In [20]:
regressor.summary()
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 60, 150)           91200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 150)           180600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 60, 150)           180600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 60, 150)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 150)              

In [21]:
regressor.fit(X_Train,Y_Train, epochs = 100, batch_size = 6)

Epoch 1/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0296
Epoch 2/100
3957/3957 [==============================] - 214s 54ms/step - loss: 0.0214
Epoch 3/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0187
Epoch 4/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0177
Epoch 5/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0178
Epoch 6/100
3957/3957 [==============================] - 216s 55ms/step - loss: 0.0177
Epoch 7/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0179
Epoch 8/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0177
Epoch 9/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0175
Epoch 10/100
3957/3957 [==============================] - 217s 55ms/step - loss: 0.0172
Epoch 11/100
3957/3957 [==============================] - 216s 55ms/step - loss: 0.0178
Epoch 12/100
3957/3957 [=================

3957/3957 [==============================] - 221s 56ms/step - loss: 0.0138
Epoch 95/100
3957/3957 [==============================] - 225s 57ms/step - loss: 0.0135
Epoch 96/100
3957/3957 [==============================] - 217s 55ms/step - loss: 0.0133
Epoch 97/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0129
Epoch 98/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0130
Epoch 99/100
3957/3957 [==============================] - 215s 54ms/step - loss: 0.0129
Epoch 100/100
3957/3957 [==============================] - 221s 56ms/step - loss: 0.0127


In [22]:
future_dates.shape

(365, 1)

In [23]:
inputs = Df_Total[len(Df_Total) - len(TestData) - 60:].values

# We need to Reshape
inputs = inputs.reshape(-1,1)

# Normalize the Dataset
inputs = sc.transform(inputs)

X_test = []
for i in range(60, 425):
    X_test.append(inputs[i-60:i])
       
# Convert into Numpy Array
X_test = np.array(X_test)

# Reshape before Passing to Network
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Pass to Model
predicted_Precip = regressor.predict(X_test)

# Do inverse Transformation to get Values
predicted_Precip = sc.inverse_transform(predicted_Precip)

In [24]:
True_Precip = 0
Predicted_Precip  = predicted_Precip
dates = TestData.index.to_list()

In [25]:
#df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=future_dates[-n_input:].index, columns=['Predictions'])
Machine_Df2 = pd.DataFrame(data={
    "Date":future_dates.index.to_list(),
    "TrueCloudcover": True_Precip,
    "PredictedCloudcover":[x[0] for x in predicted_Precip]
})

In [26]:
Machine_Df2

,Date,TrueCloudcover,PredictedCloudcover
0,2020-01-01,0,60.819454
1,2020-01-02,0,36.838741
2,2020-01-03,0,29.560249
3,2020-01-04,0,25.365686
4,2020-01-05,0,22.907373
...,...,...,...
360,2020-12-26,0,53.125656
361,2020-12-27,0,39.034477
362,2020-12-28,0,45.234097
363,2020-12-29,0,21.364910


In [32]:
#plt.figure(figsize=(25,15))
#plt.plot(TestData.index, Predicted_Precip)
#plt.show()

In [29]:
Machine_Df2.to_csv("../input/Cloudcover.csv")

In [30]:
regressor.save('../input/Cloudcover_(2020)') 

In [31]:
regressor.save('../input/Cloudcover_(2020).h5') 